In [1]:
# start SparkSession
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("project").getOrCreate()

In [5]:
#1. read in the merged dataset
data_df = spark.read\
  .format('csv')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load('s3://bigdataproject233/millions.csv')

In [6]:
data_df.show()

+--------------+-----+------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+
|            by|score|        time|           timestamp|               title|   type|                 url|                text|              parent|             deleted|                dead|     descendants|                  id|             ranking|
+--------------+-----+------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+
|         malyk| null|1511987746.0|2017-11-29 20:35:...|                null|comment|                null|That&#x27;s likel...|          15810293.0|                null|                null|            null|            15810594|                null|


In [7]:
data_df.count()

5642724

In [8]:
data_df.printSchema()

root
 |-- by: string (nullable = true)
 |-- score: string (nullable = true)
 |-- time: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- url: string (nullable = true)
 |-- text: string (nullable = true)
 |-- parent: string (nullable = true)
 |-- deleted: string (nullable = true)
 |-- dead: string (nullable = true)
 |-- descendants: string (nullable = true)
 |-- id: string (nullable = true)
 |-- ranking: string (nullable = true)



In [9]:
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType

df1 = data_df.select(data_df['by'],
                     data_df['score'].cast(DoubleType()),
                     #data_df["time"],
                     data_df['timestamp'].cast('timestamp'),
                     data_df['title'],
                     data_df['type'],
                     data_df['url'],
                     data_df['text'],
                     data_df['parent'],
                     #data_df['deleted'],
                     #data_df['dead'],
                     data_df['descendants'],
                     data_df['id'],
                     data_df['ranking'].cast(IntegerType())
                     
                             )
